#BehaviorClassifier_Run.ipynb




##🪰 This notebook is the entry point for the **BehaviorClassifier** pipeline.

---




# 1. Prepare Environment

In [1]:
# CELL 01.1 — ENVIRONMENT
"""
Ensures NumPy/pandas/Matplotlib/**Numba** are present at compatible versions.
Installs only if too old/missing. Prints one-line version report.

Py 3.12.11
"""

import importlib, sys, platform
from importlib import metadata
from packaging.version import parse as V

# Py3.12-safe baseline (matches Colab's stable stack)
TARGET = {"numpy":"2.0.2","pandas":"2.2.2","matplotlib":"3.10.0","numba":"0.60.0"}
FLOOR  = {"numpy":"2.0.0","pandas":"2.2.0","matplotlib":"3.8.0","numba":"0.60.0"}

def ver(pkg):
    try: return metadata.version(pkg)
    except metadata.PackageNotFoundError: return None

need = [p for p in FLOOR if (ver(p) is None) or (V(ver(p)) < V(FLOOR[p]))]
if need:
    %pip install -q numpy=={TARGET["numpy"]} pandas=={TARGET["pandas"]} matplotlib=={TARGET["matplotlib"]} numba=={TARGET["numba"]}
    importlib.invalidate_caches()

import numpy as np, pandas as pd, matplotlib as mpl, numba as nb
print(f"Py {sys.version.split()[0]} | NumPy {np.__version__} | pandas {pd.__version__} | mpl {mpl.__version__} | numba {nb.__version__} | {platform.system()}")

Py 3.12.11 | NumPy 2.0.2 | pandas 2.2.2 | mpl 3.10.0 | numba 0.60.0 | Linux


In [2]:
# CELL 01.2 — COLAB MOUNT & PATH SHIM
"""
Mount Google Drive and add the Codes repo to sys.path.
Fixed location by design; prints a one-line confirmation.
"""

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

from pathlib import Path
import sys

CODES = Path("/content/drive/MyDrive/1000DB/Codes")

assert CODES.exists(), f"Codes folder not found: {CODES}"
# (Optional sanity checks; keep lightweight)
assert (CODES / "BehaviorClassifier").exists(), "BehaviorClassifier package not found in Codes/"
assert (CODES / "Config").exists(), "Config package not found in Codes/"

if str(CODES) not in sys.path:
	sys.path.insert(0, str(CODES))

print(f"Using Codes root: {CODES}")

Mounted at /content/drive
Using Codes root: /content/drive/MyDrive/1000DB/Codes


In [3]:
# CELL 01.3 — CENTRALIZED IMPORTS
"""
Import read-only registries (PATH, EXPERIMENT) and the Colab/staging surface.
For silence on first run, force a non-interactive Matplotlib backend before import,
so COLOR's module-level plt.show() produces no inline output.
"""

import os, io, contextlib, importlib

# Force non-interactive backend BEFORE any matplotlib import happens
os.environ.setdefault("MPLBACKEND", "Agg")

_buf = io.StringIO()
with contextlib.redirect_stdout(_buf), contextlib.redirect_stderr(_buf):
    from Config import PATH, EXPERIMENT           # read-only registries
    import Config                                 # used later for PATH flip
    from BehaviorClassifier import BC_COLAB       # staging adapter (safe pre-flip)

In [4]:
# CELL 01.4 — DISCOVER & VALIDATE INPUTS
"""
Resolve canonical Drive paths from config and validate inputs.
POSE_SCORING comes from EXPERIMENT; no overrides here.
"""

# Resolve Drive locations (inputs + BehaviorClassification outputs)
drive_root, drive_paths = BC_COLAB["load_configs"]()  # uses Config.PATH by default

# Validate inputs on Drive (Tracked always; SLEAP only if POSE_SCORING=True)
pose_scoring = BC_COLAB["validate_inputs"](drive_paths, verbose=True)

# Tiny context print (kept minimal)
print(f"Experiment root: {drive_root}")

Validation OK. pose_scoring=True
Experiment root: /content/drive/MyDrive/1000DB


In [5]:
# CELL 01.5 — LOCAL MIRRORS & MIXED PATH
"""
Create /content mirrors for BehaviorClassification outputs and build a PATH
namespace where inputs=Drive and outputs=/content (Mixed mode).
"""

# /content mirrors for outputs (idempotent; no copies here)
local_paths = BC_COLAB["local_mirrors"](drive_root, drive_paths, verbose=True)

# PATH for the run: inputs → Drive, outputs → /content mirrors
RUNTIME_PATH = BC_COLAB["make_mixed_path_namespace"](PATH, drive_paths, local_paths)

print(f"Mixed PATH ready → outputs at: {local_paths.local_root}")

Local output mirrors ready:
  Scored  -> /content/exp_runs/1000DB/BehaviorClassification/Scored
  Pose    -> /content/exp_runs/1000DB/BehaviorClassification/Pose
  Error   -> /content/exp_runs/1000DB/BehaviorClassification/Error
  Flag    -> /content/exp_runs/1000DB/BehaviorClassification/Flag
Mixed PATH ready → outputs at: /content/exp_runs/1000DB/BehaviorClassification


In [6]:
# CELL 01.6 — FLIP PATH & BIND PACKAGE SURFACES
"""
Flip Config.PATH to Mixed, fully refresh the BehaviorClassification package so
public surfaces bind against Mixed PATH, then import package-level surfaces.
"""

import sys, Config

# 1) Flip PATH → Mixed (keep original for optional restore)
ORIGINAL_PATH = Config.PATH
Config.PATH = RUNTIME_PATH

# 2) Purge package and all its submodules so re-imports bind to Mixed PATH
for mod in [m for m in list(sys.modules) if m == "BehaviorClassifier" or m.startswith("BehaviorClassifier.")]:
    sys.modules.pop(mod, None)

# 3) Import public surfaces (lazy exports now load fresh against Mixed PATH)
from BehaviorClassifier import BC_COLAB, BC_UTILS, BC_QC, BC_MAIN

print("PATH→Mixed; BehaviorClassifier freshly bound (BC_COLAB/BC_UTILS/BC_QC/BC_MAIN).")

PATH→Mixed; BehaviorClassifier freshly bound (BC_COLAB/BC_UTILS/BC_QC/BC_MAIN).


#2. Behavior Classification

In [8]:
BATCH_SIZE = 50  # tune throughput vs. risk (larger → fewer syncs, bigger unsynced window)

BC_COLAB["start_background_sync"](
	local_paths,
	drive_paths,
	pose_scoring,
	batch_size=BATCH_SIZE,
)

result = BC_MAIN["behavior_classifier_main"]()


BC_COLAB["stop_background_sync"]()
BC_COLAB["sync_outputs_back"](
	local_paths, drive_paths, pose_scoring,
	scoring_seconds=float(result.get("scoring_seconds", 0.0)),
  )


============================= SCORING SESSION =============================

  PROCESSING    -------------------------  /content/drive/MyDrive/1000DB
  POSE SCORING  --------------------------------------------------  True

  FILES FOUND  ---------------------------------------------------  1132
  TO SCORE     ----------------------------------------------------  187
  SKIPPING     ----------------------------------------------------  945
               ---------------  scored: 0  ---  errors: 0  ---  flags: 0







============================= SESSION SUMMARY =============================

  FILES FOUND      -----------------------------------------------  1132
  FILES PROCESSED  ------------------------------------------------  187
  FILES SCORED     ------------------------------------------------  178


  QUALITY CHECK  ------------------------  |  SESSION  |---|  GLOBAL   |
  ERRORS  -------------------------------  |  0 (0%)   |---|  13 (1%)  |
  -----   error reading file-------------  |     0     |---|     2     |
  -----   wrong stim count---------------  |     0     |---|     9     |
  -----   wrong stimulus duration--------  |     0     |---|     0     |
  -----   timeline misaligned------------  |     0     |---|     2     |
  -----   many centroid NaNs-------------  |     0     |---|     0     |
  -----   missing sleap file-------------  |     0     |---|     0     |
  -----   tracked/sleap length mismatch--  |     0     |---|     0     |

  FLAGS   -------------------------------